In [1]:
# Run this cell to configure the Google Colab runtime

import os
try:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir(next((root for root, _, files in os.walk(".") if "dsait4090_project_location" in files), "."))
    print(f'Google Colab: {os.getcwd()}')
except ImportError:
    print(f'Local: {os.getcwd()}')

Local: /Users/szymong/tud/nlp/fact-checking/notebooks


In [1]:
%load_ext autoreload
%autoreload

from src.early_stopping import EarlyStopping
from src.common import get_device, read_json, DATA_PATH
from src.quantemp_processor import QuantempProcessor, QT_VERACITY_LABELS
from src.models.gpt2 import Gpt2Tokenizer, Gpt2Classifier
from src.classification_training import ClassificationTraining

import time
import torch
import logging
import os
from torch import nn

# os.environ["PYTORCH_MPS_HIGH_WATERMARK_RATIO"] = "0.0"
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

device = get_device()

/Users/szymong/.virtualenvs/tud-nlp-fact-checking/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


MPS: 1


In [2]:
train_data = read_json(f"{DATA_PATH}/raw_data/train_claims_quantemp.json")
val_data = read_json(f"{DATA_PATH}/raw_data/val_claims_quantemp.json")

limit = 20
train_data = train_data[:limit]
val_data = val_data[:limit]

In [3]:
%autoreload

gpt2_tokenizer = Gpt2Tokenizer()
data_processor = QuantempProcessor(gpt2_tokenizer, claim_decomposition=False)

train_dataset = data_processor.transform(train_data)
val_dataset = data_processor.transform(val_data)

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
%autoreload

model = Gpt2Classifier("gpt2", len(QT_VERACITY_LABELS), mlp_dim=1024, dropout=0.1).to(device)

In [5]:
%autoreload

training = ClassificationTraining(
    model_name="gpt2/basic",
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    model=model,
    optimizer=torch.optim.AdamW(model.parameters(), lr=2e-5, eps=1e-8),
    loss_function=nn.CrossEntropyLoss(),
    batch_size=4,
    early_stopping=EarlyStopping(patience=3),
    device=device,
    random_state=0
)

training.train(epochs=10)

Starting new training

EPOCH 1


train:   0%|          | 0/5 [00:00<?, ?it/s]

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.200
     eval accuracy: 0.800

    avg train loss: 1.111
     avg eval loss: 0.965

Saved checkpoint to ../models/gpt2/basic/checkpoints/01_acc_800

EPOCH 2


train:   0%|          | 0/5 [00:00<?, ?it/s]

...

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.200
     eval accuracy: 0.800

    avg train loss: 1.109
     avg eval loss: 0.741

Saved checkpoint to ../models/gpt2/basic/checkpoints/04_acc_800
Early stopping counter: 2/3

EPOCH 5


train:   0%|          | 0/5 [00:00<?, ?it/s]

eval:   0%|          | 0/5 [00:00<?, ?it/s]

    train accuracy: 0.300
     eval accuracy: 0.800

    avg train loss: 0.954
     avg eval loss: 0.728

Saved checkpoint to ../models/gpt2/basic/checkpoints/05_acc_800
Early stopping counter: 3/3
Saved best model (epoch 2) to: ../models/gpt2/basic/gpt2_basic
